# maxsmi
## Analysis of results

This notebook serves to analyse the results of the simulations ran on the Curta cluster.

## Prerequisites
This notebook will run under the condition that some simulations have been run, e.g.
```
(maxsmi) $ python maxsmi/full_workflow.py --task ESOL --aug-strategy-train augmentation_with_duplication --aug-nb-train 10 --aug-nb-test 10

```

Have a look at the [README](https://github.com/t-kimber/maxsmi/blob/main/README.md) page for more details.

In [2]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
import dataframe_image as dfi
from maxsmi.utils_analysis import retrieve_metric

In [3]:
# Make a folder for output figures
os.makedirs("figures", exist_ok=True)

# Dataset
We consider the following datasets:

- ESOL
- free_solv
- lipophilicity

In [4]:
# TASK = "lipophilicity"
# TASK = "ESOL"
TASK = "free_solv"

### Grid: augmentation number

The models were run of a fine grid augmentation: from 1 to 20 with a step of 1 as well as a coarser grid: from 20 to 100 with a step of 10.

In [5]:
fine_grid = [elem for elem in range(0, 21, 1)]
coarse_grid = [elem for elem in range(10, 110, 10)]

temp_grid = [elem for elem in range(30, 110, 10)]
full_grid = fine_grid + temp_grid

In [6]:
def array_by_strategy(augmentation_strategy,
                      task="ESOL",
                      set_="test",
                      metric="rmse",
                      grid=full_grid):
    """

    Returns an array of a given metric for a given task.

    Parameters
    ----------
    augmentation_strategy : str
        The augmentation strategy to consider,
        e.g. `augmentation_with_reduced_duplication`.
    task : str, default `ESOL`
        The dataset to consider: ESOL, free_solv or lipophilicity.
    set_ : str, default `test`
        The evaluation set, train or test.
    metric: str, default `rmse`
        The performance metric to consider,
        such as the root mean squared error (rmse).
    grid: list
        The augmentation number to retrieve.

    Returns
    -------
    np.array
        Numeric values for `metric`.
    """

    models = ["CONV1D", "CONV2D", "RNN"]

    result_array = np.zeros((len(grid), len(models)))

    if augmentation_strategy == "augmentation_maximum_estimation":
        if task == "ESOL":
            task = "ESOL_SMALL"
            # Estimated maximum strategy was only evaluated
            # on a subset of ESOL, because time-intensive purposes.

    for i, model in enumerate(models):
        for j, augmentation_num in enumerate(grid):
            try:
                y = retrieve_metric(
                    metric,
                    set_,
                    task,
                    augmentation_strategy,
                    augmentation_num,
                    augmentation_strategy,
                    augmentation_num,
                    model,
                )
            except FileNotFoundError:
                y = np.nan
            result_array[j, i] = y
    return result_array

## Retrieve performance by augmentation strategy

In [7]:
results_without_dupl = array_by_strategy("augmentation_without_duplication",
                                         task=TASK)
results_with_dupl = array_by_strategy("augmentation_with_duplication",
                                      task=TASK)
results_with_red_dupl = array_by_strategy("augmentation_with_reduced_\
duplication",
                                          task=TASK)
results_max_est = array_by_strategy("augmentation_maximum_estimation",
                                    task=TASK)
results_no_aug = array_by_strategy("no_augmentation",
                                   task=TASK)

_Note:_

The augmentation strategy `estimated maximum` was not evaluated on the lipohilicity dataset.

In [8]:
if TASK == "lipophilicity":
    full_results = np.concatenate([results_without_dupl,
                                   results_with_dupl,
                                   results_with_red_dupl,
                                   results_no_aug], axis=1)
    column_name = [
        ('with duplication', 'CONV1D'),
        ('with duplication', 'CONV2D'),
        ('with duplication', 'RNN'),
        ('without duplication', 'CONV1D'),
        ('without duplication', 'CONV2D'),
        ('without duplication', 'RNN'),
        ('with reduced duplication', 'CONV1D'),
        ('with reduced duplication', 'CONV2D'),
        ('with reduced duplication', 'RNN'),
        ('no augmentation', 'CONV1D'),
        ('no augmentation', 'CONV2D'),
        ('no augmentation', 'RNN'),
    ]

else:
    column_name = [
        ('with duplication', 'CONV1D'),
        ('with duplication', 'CONV2D'),
        ('with duplication', 'RNN'),
        ('without duplication', 'CONV1D'),
        ('without duplication', 'CONV2D'),
        ('without duplication', 'RNN'),
        ('with reduced duplication', 'CONV1D'),
        ('with reduced duplication', 'CONV2D'),
        ('with reduced duplication', 'RNN'),
        ('no augmentation', 'CONV1D'),
        ('no augmentation', 'CONV2D'),
        ('no augmentation', 'RNN'),
        ('estimated maximum', 'CONV1D'),
        ('estimated maximum', 'CONV2D'),
        ('estimated maximum', 'RNN'),
    ]
    full_results = np.concatenate([results_without_dupl,
                                   results_with_dupl,
                                   results_with_red_dupl,
                                   results_no_aug,
                                   results_max_est], axis=1)

## Table

Generate the pandas dataframe with results, with multicolumns.

In [9]:
dataframe = pd.DataFrame(full_results,
                         columns=column_name,
                         index=full_grid)

In [10]:
dataframe.columns = pd.MultiIndex.from_tuples(dataframe.columns,
                                              names=['Strategy', 'Model'])
dataframe

Strategy with duplication                     without duplication            \
Model              CONV1D    CONV2D       RNN              CONV1D    CONV2D   
0                     NaN       NaN       NaN                 NaN       NaN   
1                2.576553  2.867690  2.513789            2.603028  2.881221   
2                2.107822  2.222457  1.989296            2.021471  2.102961   
3                1.914096  2.315347  1.900420            1.955804  1.966772   
4                1.494451  1.646310  1.545067            1.444198  1.692500   
5                1.260051  1.524381  1.380570            1.341978  1.516899   
6                1.306289  1.580351  1.397609            1.277238  1.722576   
7                1.415770  2.004465  1.611623            1.364584  1.492614   
8                1.257534  1.446213  1.273156            1.281537  1.472069   
9                1.325040  1.578069  1.237859            1.323704  1.598498   
10               1.190051  1.485084  1.303380            1.231074  1.517179   
11               1.224704  1.433660  1.331317            1.195230  1.518891   
12               1.181272  1.392629  1.422093            1.197999  1.438357   
13               1.307932  1.522008  1.391981            1.146017  1.584459   
14               1.161572  1.483182  1.239547            1.196892  1.500683   
15               1.168624  1.516332  1.237257            1.160298  1.545302   
16               1.138419  1.305616  1.256118            1.095630  1.488440   
17               1.146968  1.493314  1.320557            1.161985  2.287752   
18               1.139168  1.503643  1.259671            1.140851  1.493708   
19               1.163891  1.475479  1.244214            1.159093  1.561230   
20               1.139182  1.522832  1.329581            1.084603  1.449123   
30               1.082927  1.374254  1.299155            1.120433  1.552755   
40               1.140160  1.524841  1.311394            1.051448  1.700225   
50               1.091181  1.579425  1.290504            1.053133  1.452142   
60               1.073911  1.553867  1.264451            1.093533  1.427343   
70               1.084567  1.679290  1.239409            1.032067  1.475831   
80               1.125119  1.638817  1.270036            1.069986  1.448608   
90               1.101427  1.506012  1.280484            1.058080  1.555853   
100              1.115065  1.630677  1.276083            1.056276  1.598153   

Strategy           with reduced duplication                      \
Model          RNN                   CONV1D    CONV2D       RNN   
0              NaN                      NaN       NaN       NaN   
1         2.395808                 2.691229  3.017233  2.431184   
2         2.122376                 2.090806  1.978924  2.001960   
3         1.809600                 1.916622  2.371182  4.111752   
4         1.479107                 1.420278  1.594135  1.568063   
5         1.332053                 1.278124  1.478080  1.468778   
6         1.351626                 1.305634  1.655896  1.357706   
7         1.445625                 1.415027  1.558877  1.398226   
8         1.288066                 1.237856  1.374845  1.258589   
9         1.240258                 1.286977  1.497929  1.252765   
10        1.290062                 1.211134  1.543577  1.228639   
11        1.224318                 1.238219  1.410347  1.282998   
12        1.273001                 1.196125  1.448572  1.198974   
13        1.326868                 1.199066  1.556534  1.266257   
14        1.278678                 1.149466  1.501389  1.477429   
15        1.202259                 1.188089  1.441458  1.222893   
16        1.156832                 1.101718  1.397177  1.249520   
17        1.231283                 1.116592  1.501469  1.294920   
18        1.569040                 1.098654  1.572073  1.213241   
19        1.530544                 1.128689  1.557709  1.272404   
20        1.273992                 1.108972  1.501850  1.198534   
30       

## Cosmetics

We beautify the dataframe for readibily.

In [11]:
def color_nan_white(val):
    """Color the nan text white"""
    if np.isnan(val):
        return 'color: white'

In [12]:
def color_nan_white_background(val):
    """Color the nan cell background white"""
    if np.isnan(val):
        return 'background-color: white'

In [13]:
df_styler = dataframe.style.\
    set_caption(f"Data: {TASK}").\
    format("{:.3f}").\
    background_gradient(cmap='Purples', axis=None).\
    applymap(lambda x: color_nan_white(x)).\
    applymap(lambda x: color_nan_white_background(x)).\
    highlight_min(color="yellow", axis=None)

df_styler

In [14]:
dfi.export(df_styler, f"figures/dataframe_{TASK}.png")